In [70]:
import os


%run Metric.ipynb
%run Util.ipynb

k_up = 'up'
k_down = 'down'
k_contains = 'contains'
confirm_lines=5

clean_k_columns = list(k_columns)
clean_k_columns.append('status')

top_turning_point = 'top'
bottom_turning_point = 'bottom'

class ChanBiAnalyser:
    
    def __init__(self,symbol,level,start_date,end_date,short,mid):
        self.symbol = symbol
        self.level = level
        self.start_date = start_date
        self.end_date = end_date
        self.short = short
        self.mid = mid
        
        self.metric = Metric(self.symbol,self.level,self.short,self.mid)
        self.chan_df = self.build_chan_k()
        
        
    def get_k_status(self,cur_k,last_k):
        if(last_k is None):
            return k_up
        
        if((cur_k['high']>last_k['high']) and (cur_k['low']>last_k['low'])):
            return k_up
        elif((cur_k['high']<last_k['high']) and (cur_k['low']<last_k['low'])):
            return k_down
        
        return k_contains
    
    def process_contains_scenario(self,cur_k,last_k,chan_df):
        open_price = last_k['open']
        close = cur_k['close']
        volume = cur_k['volume']
        amount = cur_k['amount']

        last_k_status = chan_df[chan_df.shape[0]-1:]['status'].values[0]

        status = k_up
        if(last_k_status==k_up):
            high = max(cur_k['high'], last_k['high'])
            low = max(cur_k['low'], last_k['low'])
        elif(last_k_status==k_down):
            high = min(cur_k['high'], last_k['high'])
            low = min(cur_k['low'], last_k['low'])
        else:
            print('Warnging, dirty after processed K lines ' +str(row))
            
        merged_k = [cur_k['date'],open_price,high,low,close,volume,amount,last_k_status]
        return merged_k
        
        
    def build_clean_k_series(self,row,status):
        array = list(row[:7])
        array.append(status)
        clean_k = pd.Series(array,index=clean_k_columns)
        return clean_k
    
        
    def build_chan_k(self):
        part_df = self.metric.get_k_series_between(self.start_date, self.end_date)

        chan_df = pd.DataFrame(columns=clean_k_columns)
        chan_df['high'] = chan_df['high'].astype('float')
        chan_df['low'] = chan_df['low'].astype('float')
        
        first=True
        
        index=0
        for key,row in part_df.iterrows():
            if(first==True):
                chan_df.loc[index] = self.build_clean_k_series(row.values,k_up)
                index += 1
                first=False
                continue

            cur_k = row
            last_k_data = chan_df[chan_df.shape[0]-1:].values[0]
            last_k = pd.Series(last_k_data,index=clean_k_columns)
            
            k_status = self.get_k_status(cur_k,last_k)
            if(k_status==k_contains):
                merged_k = self.process_contains_scenario(cur_k,last_k,chan_df)
                merged_k_series = pd.Series(merged_k,index=clean_k_columns)
                
                chan_df.loc[index-1] = merged_k_series
            else:
                chan_df.loc[index] = self.build_clean_k_series(row.values,k_status)
                index += 1
            
        chan_df['datetime'] = pd.to_datetime(chan_df['date'])
        return chan_df
            
    def get_opposite_status(self,status):
        if(status==k_up):
            return k_down
        elif(status==k_down):
            return k_up
        else:
            print('Warning, clean_df has contains direction k lines')
        
#     未来五条线破坏你的方向,你就是转折点
    def is_turning_point(self, cur_date, bi_direction):
        latest_n=10
        cur_status = self.chan_df[self.chan_df['date']==cur_date]['status']
        
        cur_index = self.chan_df[self.chan_df['date']==cur_date].index.values[0]
        end_index = cur_index.astype(int)+latest_n

        part = self.chan_df[cur_index:end_index]

        if(part[part['status']==bi_direction].shape[0]>confirm_lines):
            return True
        else:
            return False
        
#   顶分型
    def is_top_point(self,cur_date):
        cur_series = self.chan_df[self.chan_df['date']==cur_date]
        
        cur_index = cur_series.index.values[0]
        next_clean_k = self.chan_df[cur_index+1:cur_index.astype(int)+2]
        if(len(next_clean_k)==0):
            return False
        
        if((cur_series['status'].values[0]==k_up) and (next_clean_k['status'].values[0]==k_down)):
            return True
        else:
            return False
            
        
#   底分型
    def is_bottom_point(self,cur_date):
        cur_series = self.chan_df[self.chan_df['date']==cur_date]
        
        cur_index = cur_series.index.values[0]+1
        end_index = cur_index.astype(int)+2
        
        next_clean_k = self.chan_df[cur_index:end_index]
        if(len(next_clean_k)==0):
            return False
        
        if((cur_series['status'].values[0]==k_down) and (next_clean_k['status'].values[0]==k_up)):
            return True
        else:
            return False
    
    
    def get_turning_point(self):
        turning_point_df = pd.DataFrame(columns=['symbol','date','turning_point','high','low'])
        
        for key,row in self.chan_df.iterrows():
            if(self.is_top_point(row['date'])):
                turning_point_df.loc[turning_point_df.shape[0]+1] = [self.symbol,row['date'],top_turning_point,row['high'],row['low']]
            if(self.is_bottom_point(row['date'])):
                turning_point_df.loc[turning_point_df.shape[0]+1] = [self.symbol,row['date'],bottom_turning_point,row['high'],row['low']]

        return turning_point_df
        
        
    def count_k_between(self,start_date,end_date):
        start_datetime = to_datetime(start_date)
        end_datetime = to_datetime(end_date)
        part = self.chan_df[(self.chan_df['datetime']>start_datetime) & (self.chan_df['datetime']<end_date)]
        
        return part.shape[0]
        
        
    def add_addtional_fields(self,bi_df):
        bi_df['price'] = 0
        bi_df['increase'] = 0
        bi_df = bi_df[bi_df['confirm']=="YES"]
        bi_df=bi_df.reset_index()
        
        for key,row in bi_df.iterrows():
            if(row['turning_point']==top_turning_point):
                bi_df.loc[key,'price']=row['high']
            else:
                bi_df.loc[key,'price']=row['low'] 
             
        first = True
        for key,row in bi_df.iterrows():
            if(first==True):
                bi_df.loc[key,'increase']=0
                first=False
                continue
            
            last_point = bi_df.loc[key-1]

            increase_per = round(100*(row['price']-last_point['price'])/last_point['price'],2)
            bi_df.loc[key,'increase']=increase_per
            
        return bi_df
        
        
    def build_chan_bi_file_path(self):
        return chan_bi_dir + 'chan_bi'+'_'+self.symbol+'_'+str(self.short)+'_'+str(self.mid)+'_'+self.start_date+'_'+self.end_date+'_'+chan_bi_version+'.csv'
    
        
    def build_chan_bi_df(self):
        bi_df = self.get_turning_point()
        bi_df['confirm'] ='NO'
        
        first=True
        direction=''
        last_confirm = ''
        last_confirm_index=0
        for key,row in bi_df.iterrows():
            if(first==True):
                direction = row['turning_point']
                bi_df.loc[key,'confirm'] = 'YES'
                last_confirm = row
                last_confirm_index = key
                first=False
                continue
            
            cur_point = row
            last_point = bi_df.loc[key-1]
            
            
            if(cur_point['turning_point']!=last_confirm['turning_point']):
                count_k = self.count_k_between(last_confirm['date'],cur_point['date'])

                if(count_k<3):
                    row['confirm'] = 'NO'
                else:
                    bi_df.loc[key,'confirm'] = 'YES'
                    
                    last_confirm = row
                    last_confirm_index = key
            
            if(cur_point['turning_point']==last_confirm['turning_point']):
                if(cur_point['turning_point']==top_turning_point):
                    if(cur_point['high']>last_confirm['high']):
                        bi_df.loc[key,'confirm'] = 'YES'
                        bi_df.loc[last_confirm_index,'confirm']='NO'
                        last_confirm = cur_point
                        last_confirm_index = key
                    else:
                        cur_point['confirm']='NO'
                else:
                    if(cur_point['low']<last_confirm['low']):
                        bi_df.loc[key,'confirm'] = 'YES'
                        bi_df.loc[last_confirm_index,'confirm']='NO'
                        last_confirm = cur_point
                        last_confirm_index = key
                    else:
                        cur_point['confirm']='NO'
        
        return self.add_addtional_fields(bi_df)
        
        
    def get_chan_bi(self):
        file_path = self.build_chan_bi_file_path()
        
        chan_bi_df = None
        if(os.path.exists(file_path)):
            chan_bi_df = pd.read_csv(file_path)
        else:
            chan_bi_df = self.build_chan_bi_df()
                    
            file_path = self.build_chan_bi_file_path()
            chan_bi_df.to_csv(file_path, index=0)

        return chan_bi_df
        